<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px\" align="left">

# Purity Randomized Benchmarking

In [1]:
#Import the RB Functions
import qiskit.ignis.verification.randomized_benchmarking as rb

#Import Qiskit classes classes
import qiskit
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error, thermal_relaxation_error


In [2]:
# Example of 1-qubit Purity RB
#Number of qubits
nQ = 3
#There are 3 qubits: Q0,Q1,Q2.
#Number of seeds (random sequences)
nseeds = 5
#Number of Cliffords in the sequence (start, stop, steps)
nCliffs = np.arange(1,20,2)
print (len(nCliffs))
#2Q RB on Q0,Q2 and 1Q RB on Q1
rb_pattern = [[0],[1],[2]]

10


In [3]:
rb_opts = {}
rb_opts['length_vector'] = nCliffs
rb_opts['nseeds'] = nseeds
rb_opts['rb_pattern'] = rb_pattern
rb_opts['is_purity'] = True
rb_purity_circs, xdata, npurity = rb.randomized_benchmarking_seq(**rb_opts)
print (npurity)

3


In [4]:
for i in range(len(rb_purity_circs)):
    for j in range(len(rb_purity_circs[i])):
        for k in range(len(rb_purity_circs[i][j])):
            print (i,j,k, rb_purity_circs[i][j][k].name)

0 0 0 rb_purity_Z_length_0_seed_0
0 0 1 rb_purity_Z_length_1_seed_0
0 0 2 rb_purity_Z_length_2_seed_0
0 0 3 rb_purity_Z_length_3_seed_0
0 0 4 rb_purity_Z_length_4_seed_0
0 0 5 rb_purity_Z_length_5_seed_0
0 0 6 rb_purity_Z_length_6_seed_0
0 0 7 rb_purity_Z_length_7_seed_0
0 0 8 rb_purity_Z_length_8_seed_0
0 0 9 rb_purity_Z_length_9_seed_0
0 1 0 rb_purity_X_length_0_seed_0
0 1 1 rb_purity_X_length_1_seed_0
0 1 2 rb_purity_X_length_2_seed_0
0 1 3 rb_purity_X_length_3_seed_0
0 1 4 rb_purity_X_length_4_seed_0
0 1 5 rb_purity_X_length_5_seed_0
0 1 6 rb_purity_X_length_6_seed_0
0 1 7 rb_purity_X_length_7_seed_0
0 1 8 rb_purity_X_length_8_seed_0
0 1 9 rb_purity_X_length_9_seed_0
0 2 0 rb_purity_Y_length_0_seed_0
0 2 1 rb_purity_Y_length_1_seed_0
0 2 2 rb_purity_Y_length_2_seed_0
0 2 3 rb_purity_Y_length_3_seed_0
0 2 4 rb_purity_Y_length_4_seed_0
0 2 5 rb_purity_Y_length_5_seed_0
0 2 6 rb_purity_Y_length_6_seed_0
0 2 7 rb_purity_Y_length_7_seed_0
0 2 8 rb_purity_Y_length_8_seed_0
0 2 9 rb_purit

In [5]:
print (rb_purity_circs[0][0][0])

         ┌───┐┌─────┐┌───┐┌───┐  ░     ┌───┐┌───┐┌───┐┌───┐┌─┐   
qr_0: |0>┤ H ├┤ Sdg ├┤ H ├┤ Y ├──░─────┤ Y ├┤ H ├┤ S ├┤ H ├┤M├───
         ├───┤└┬───┬┘└─░─┘├───┤┌───┐┌─┐└───┘└───┘└───┘└───┘└╥┘   
qr_1: |0>┤ H ├─┤ X ├───░──┤ X ├┤ H ├┤M├─────────────────────╫────
         ├───┤┌┴───┴┐┌───┐├───┤└─░─┘└╥┘┌───┐┌───┐┌───┐┌───┐ ║ ┌─┐
qr_2: |0>┤ H ├┤ Sdg ├┤ H ├┤ Z ├──░───╫─┤ Z ├┤ H ├┤ S ├┤ H ├─╫─┤M├
         └───┘└─────┘└───┘└───┘  ░   ║ └───┘└───┘└───┘└───┘ ║ └╥┘
 cr_0: 0 ════════════════════════════╬══════════════════════╩══╬═
                                     ║                         ║ 
 cr_1: 0 ════════════════════════════╩═════════════════════════╬═
                                                               ║ 
 cr_2: 0 ══════════════════════════════════════════════════════╩═
                                                                 


In [6]:
print (rb_purity_circs[0][1][0])

         ┌───┐┌─────┐┌───┐┌───┐  ░      ┌───┐        ┌───┐┌───┐┌───┐»
qr_0: |0>┤ H ├┤ Sdg ├┤ H ├┤ Y ├──░──────┤ Y ├────────┤ H ├┤ S ├┤ H ├»
         ├───┤└┬───┬┘└─░─┘├───┤┌───┐┌───┴───┴────┐┌─┐└───┘└───┘└───┘»
qr_1: |0>┤ H ├─┤ X ├───░──┤ X ├┤ H ├┤ Rx(1.5708) ├┤M├───────────────»
         ├───┤┌┴───┴┐┌───┐├───┤└─░─┘└───┬───┬────┘└╥┘┌───┐┌───┐┌───┐»
qr_2: |0>┤ H ├┤ Sdg ├┤ H ├┤ Z ├──░──────┤ Z ├──────╫─┤ H ├┤ S ├┤ H ├»
         └───┘└─────┘└───┘└───┘  ░      └───┘      ║ └───┘└───┘└───┘»
 cr_0: 0 ══════════════════════════════════════════╬════════════════»
                                                   ║                »
 cr_1: 0 ══════════════════════════════════════════╩════════════════»
                                                                    »
 cr_2: 0 ═══════════════════════════════════════════════════════════»
                                                                    »
«      ┌────────────┐┌─┐   
«qr_0: ┤ Rx(1.5708) ├┤M├───
«      └────────────┘└╥┘   
«qr_1:

In [7]:
print (rb_purity_circs[0][2][0])

         ┌───┐┌─────┐┌───┐┌───┐  ░      ┌───┐        ┌───┐┌───┐┌───┐»
qr_0: |0>┤ H ├┤ Sdg ├┤ H ├┤ Y ├──░──────┤ Y ├────────┤ H ├┤ S ├┤ H ├»
         ├───┤└┬───┬┘└─░─┘├───┤┌───┐┌───┴───┴────┐┌─┐└───┘└───┘└───┘»
qr_1: |0>┤ H ├─┤ X ├───░──┤ X ├┤ H ├┤ Ry(1.5708) ├┤M├───────────────»
         ├───┤┌┴───┴┐┌───┐├───┤└─░─┘└───┬───┬────┘└╥┘┌───┐┌───┐┌───┐»
qr_2: |0>┤ H ├┤ Sdg ├┤ H ├┤ Z ├──░──────┤ Z ├──────╫─┤ H ├┤ S ├┤ H ├»
         └───┘└─────┘└───┘└───┘  ░      └───┘      ║ └───┘└───┘└───┘»
 cr_0: 0 ══════════════════════════════════════════╬════════════════»
                                                   ║                »
 cr_1: 0 ══════════════════════════════════════════╩════════════════»
                                                                    »
 cr_2: 0 ═══════════════════════════════════════════════════════════»
                                                                    »
«      ┌────────────┐┌─┐   
«qr_0: ┤ Ry(1.5708) ├┤M├───
«      └────────────┘└╥┘   
«qr_1:

In [8]:
# Example of 2-qubits Purity RB
#Number of qubits
nQ = 4
#Number of seeds (random sequences)
nseeds = 3
#Number of Cliffords in the sequence (start, stop, steps)
nCliffs = np.arange(1,10,2)
print (len(nCliffs))
#2Q RB on Q0,Q2 and 1Q RB on Q1
rb_pattern = [[0,1],[2,3]]

5


In [9]:
rb_opts = {}
rb_opts['length_vector'] = nCliffs
rb_opts['nseeds'] = nseeds
rb_opts['rb_pattern'] = rb_pattern
rb_opts['is_purity'] = True
rb_purity_circs, xdata, npurity = rb.randomized_benchmarking_seq(**rb_opts)
print (npurity)

9


In [10]:
for i in range(len(rb_purity_circs)):
    for j in range(len(rb_purity_circs[i])):
        for k in range(len(rb_purity_circs[i][j])):
            print (i,j,k, rb_purity_circs[i][j][k].name)

0 0 0 rb_purity_ZZ_length_0_seed_0
0 0 1 rb_purity_ZZ_length_1_seed_0
0 0 2 rb_purity_ZZ_length_2_seed_0
0 0 3 rb_purity_ZZ_length_3_seed_0
0 0 4 rb_purity_ZZ_length_4_seed_0
0 1 0 rb_purity_XZ_length_0_seed_0
0 1 1 rb_purity_XZ_length_1_seed_0
0 1 2 rb_purity_XZ_length_2_seed_0
0 1 3 rb_purity_XZ_length_3_seed_0
0 1 4 rb_purity_XZ_length_4_seed_0
0 2 0 rb_purity_YZ_length_0_seed_0
0 2 1 rb_purity_YZ_length_1_seed_0
0 2 2 rb_purity_YZ_length_2_seed_0
0 2 3 rb_purity_YZ_length_3_seed_0
0 2 4 rb_purity_YZ_length_4_seed_0
0 3 0 rb_purity_ZX_length_0_seed_0
0 3 1 rb_purity_ZX_length_1_seed_0
0 3 2 rb_purity_ZX_length_2_seed_0
0 3 3 rb_purity_ZX_length_3_seed_0
0 3 4 rb_purity_ZX_length_4_seed_0
0 4 0 rb_purity_XX_length_0_seed_0
0 4 1 rb_purity_XX_length_1_seed_0
0 4 2 rb_purity_XX_length_2_seed_0
0 4 3 rb_purity_XX_length_3_seed_0
0 4 4 rb_purity_XX_length_4_seed_0
0 5 0 rb_purity_YX_length_0_seed_0
0 5 1 rb_purity_YX_length_1_seed_0
0 5 2 rb_purity_YX_length_2_seed_0
0 5 3 rb_purity_YX_l

In [11]:
print (rb_purity_circs[0][0][0])

         ┌─────┐ ┌───┐      ┌───┐   ░   ┌───┐     ┌───┐   ┌───┐┌─┐            »
qr_0: |0>┤ Sdg ├─┤ H ├───■──┤ Z ├───░───┤ Z ├──■──┤ H ├───┤ S ├┤M├────────────»
         └┬───┬┘ └───┘ ┌─┴─┐└───┘   ░   └───┘┌─┴─┐├───┤┌─┐└───┘└╥┘            »
qr_1: |0>─┤ H ├────────┤ X ├────────░────────┤ X ├┤ H ├┤M├──────╫─────────────»
          ├───┤ ┌─────┐├───┤     ┌─────┐┌───┐├───┤└─░─┘└╥┘┌───┐ ║  ┌───┐ ┌───┐»
qr_2: |0>─┤ H ├─┤ Sdg ├┤ H ├──■──┤ Sdg ├┤ H ├┤ Z ├──░───╫─┤ Z ├─╫──┤ H ├─┤ S ├»
         ┌┴───┴┐└┬───┬┘└───┘┌─┴─┐└┬───┬┘├───┤├───┤  ░   ║ ├───┤ ║ ┌┴───┴┐├───┤»
qr_3: |0>┤ Sdg ├─┤ H ├──────┤ X ├─┤ H ├─┤ S ├┤ Z ├──░───╫─┤ Z ├─╫─┤ Sdg ├┤ H ├»
         └─────┘ └───┘      └───┘ └───┘ └───┘└───┘  ░   ║ └───┘ ║ └─────┘└───┘»
 cr_0: 0 ═══════════════════════════════════════════════╬═══════╩═════════════»
                                                        ║                     »
 cr_1: 0 ═══════════════════════════════════════════════╩═════════════════════»
                                        

In [12]:
print (rb_purity_circs[0][1][0])

         ┌─────┐ ┌───┐      ┌───┐   ░   ┌───┐     ┌───┐   ┌───┐┌────────────┐»
qr_0: |0>┤ Sdg ├─┤ H ├───■──┤ Z ├───░───┤ Z ├──■──┤ H ├───┤ S ├┤ Rx(1.5708) ├»
         └┬───┬┘ └───┘ ┌─┴─┐└───┘   ░   └───┘┌─┴─┐├───┤┌─┐└───┘└────────────┘»
qr_1: |0>─┤ H ├────────┤ X ├────────░────────┤ X ├┤ H ├┤M├───────────────────»
          ├───┤ ┌─────┐├───┤     ┌─────┐┌───┐├───┤└─░─┘└╥┘┌───┐    ┌───┐     »
qr_2: |0>─┤ H ├─┤ Sdg ├┤ H ├──■──┤ Sdg ├┤ H ├┤ Z ├──░───╫─┤ Z ├────┤ H ├─────»
         ┌┴───┴┐└┬───┬┘└───┘┌─┴─┐└┬───┬┘├───┤├───┤  ░   ║ ├───┤   ┌┴───┴┐    »
qr_3: |0>┤ Sdg ├─┤ H ├──────┤ X ├─┤ H ├─┤ S ├┤ Z ├──░───╫─┤ Z ├───┤ Sdg ├────»
         └─────┘ └───┘      └───┘ └───┘ └───┘└───┘  ░   ║ └───┘   └─────┘    »
 cr_0: 0 ═══════════════════════════════════════════════╬════════════════════»
                                                        ║                    »
 cr_1: 0 ═══════════════════════════════════════════════╩════════════════════»
                                                    

In [13]:
print (rb_purity_circs[0][2][0])

         ┌─────┐ ┌───┐      ┌───┐   ░   ┌───┐     ┌───┐   ┌───┐┌────────────┐»
qr_0: |0>┤ Sdg ├─┤ H ├───■──┤ Z ├───░───┤ Z ├──■──┤ H ├───┤ S ├┤ Ry(1.5708) ├»
         └┬───┬┘ └───┘ ┌─┴─┐└───┘   ░   └───┘┌─┴─┐├───┤┌─┐└───┘└────────────┘»
qr_1: |0>─┤ H ├────────┤ X ├────────░────────┤ X ├┤ H ├┤M├───────────────────»
          ├───┤ ┌─────┐├───┤     ┌─────┐┌───┐├───┤└─░─┘└╥┘┌───┐    ┌───┐     »
qr_2: |0>─┤ H ├─┤ Sdg ├┤ H ├──■──┤ Sdg ├┤ H ├┤ Z ├──░───╫─┤ Z ├────┤ H ├─────»
         ┌┴───┴┐└┬───┬┘└───┘┌─┴─┐└┬───┬┘├───┤├───┤  ░   ║ ├───┤   ┌┴───┴┐    »
qr_3: |0>┤ Sdg ├─┤ H ├──────┤ X ├─┤ H ├─┤ S ├┤ Z ├──░───╫─┤ Z ├───┤ Sdg ├────»
         └─────┘ └───┘      └───┘ └───┘ └───┘└───┘  ░   ║ └───┘   └─────┘    »
 cr_0: 0 ═══════════════════════════════════════════════╬════════════════════»
                                                        ║                    »
 cr_1: 0 ═══════════════════════════════════════════════╩════════════════════»
                                                    

In [14]:
print (rb_purity_circs[0][3][0])

         ┌─────┐ ┌───┐      ┌───┐   ░   ┌───┐     ┌───┐    ┌───┐     ┌─┐   »
qr_0: |0>┤ Sdg ├─┤ H ├───■──┤ Z ├───░───┤ Z ├──■──┤ H ├────┤ S ├─────┤M├───»
         └┬───┬┘ └───┘ ┌─┴─┐└───┘   ░   └───┘┌─┴─┐├───┤┌───┴───┴────┐└╥┘┌─┐»
qr_1: |0>─┤ H ├────────┤ X ├────────░────────┤ X ├┤ H ├┤ Rx(1.5708) ├─╫─┤M├»
          ├───┤ ┌─────┐├───┤     ┌─────┐┌───┐├───┤└─░─┘└───┬───┬────┘ ║ └╥┘»
qr_2: |0>─┤ H ├─┤ Sdg ├┤ H ├──■──┤ Sdg ├┤ H ├┤ Z ├──░──────┤ Z ├──────╫──╫─»
         ┌┴───┴┐└┬───┬┘└───┘┌─┴─┐└┬───┬┘├───┤├───┤  ░      ├───┤      ║  ║ »
qr_3: |0>┤ Sdg ├─┤ H ├──────┤ X ├─┤ H ├─┤ S ├┤ Z ├──░──────┤ Z ├──────╫──╫─»
         └─────┘ └───┘      └───┘ └───┘ └───┘└───┘  ░      └───┘      ║  ║ »
 cr_0: 0 ═════════════════════════════════════════════════════════════╩══╬═»
                                                                         ║ »
 cr_1: 0 ════════════════════════════════════════════════════════════════╩═»
                                                                           »

In [15]:
print (rb_purity_circs[0][4][0])

         ┌─────┐ ┌───┐      ┌───┐   ░   ┌───┐     ┌───┐    ┌───┐        »
qr_0: |0>┤ Sdg ├─┤ H ├───■──┤ Z ├───░───┤ Z ├──■──┤ H ├────┤ S ├────────»
         └┬───┬┘ └───┘ ┌─┴─┐└───┘   ░   └───┘┌─┴─┐├───┤┌───┴───┴────┐┌─┐»
qr_1: |0>─┤ H ├────────┤ X ├────────░────────┤ X ├┤ H ├┤ Rx(1.5708) ├┤M├»
          ├───┤ ┌─────┐├───┤     ┌─────┐┌───┐├───┤└─░─┘└───┬───┬────┘└╥┘»
qr_2: |0>─┤ H ├─┤ Sdg ├┤ H ├──■──┤ Sdg ├┤ H ├┤ Z ├──░──────┤ Z ├──────╫─»
         ┌┴───┴┐└┬───┬┘└───┘┌─┴─┐└┬───┬┘├───┤├───┤  ░      ├───┤      ║ »
qr_3: |0>┤ Sdg ├─┤ H ├──────┤ X ├─┤ H ├─┤ S ├┤ Z ├──░──────┤ Z ├──────╫─»
         └─────┘ └───┘      └───┘ └───┘ └───┘└───┘  ░      └───┘      ║ »
 cr_0: 0 ═════════════════════════════════════════════════════════════╬═»
                                                                      ║ »
 cr_1: 0 ═════════════════════════════════════════════════════════════╩═»
                                                                        »
 cr_2: 0 ═════════════════════════════

In [16]:
print (rb_purity_circs[0][5][0])

         ┌─────┐ ┌───┐      ┌───┐   ░   ┌───┐     ┌───┐    ┌───┐        »
qr_0: |0>┤ Sdg ├─┤ H ├───■──┤ Z ├───░───┤ Z ├──■──┤ H ├────┤ S ├────────»
         └┬───┬┘ └───┘ ┌─┴─┐└───┘   ░   └───┘┌─┴─┐├───┤┌───┴───┴────┐┌─┐»
qr_1: |0>─┤ H ├────────┤ X ├────────░────────┤ X ├┤ H ├┤ Rx(1.5708) ├┤M├»
          ├───┤ ┌─────┐├───┤     ┌─────┐┌───┐├───┤└─░─┘└───┬───┬────┘└╥┘»
qr_2: |0>─┤ H ├─┤ Sdg ├┤ H ├──■──┤ Sdg ├┤ H ├┤ Z ├──░──────┤ Z ├──────╫─»
         ┌┴───┴┐└┬───┬┘└───┘┌─┴─┐└┬───┬┘├───┤├───┤  ░      ├───┤      ║ »
qr_3: |0>┤ Sdg ├─┤ H ├──────┤ X ├─┤ H ├─┤ S ├┤ Z ├──░──────┤ Z ├──────╫─»
         └─────┘ └───┘      └───┘ └───┘ └───┘└───┘  ░      └───┘      ║ »
 cr_0: 0 ═════════════════════════════════════════════════════════════╬═»
                                                                      ║ »
 cr_1: 0 ═════════════════════════════════════════════════════════════╩═»
                                                                        »
 cr_2: 0 ═════════════════════════════

In [17]:
print (rb_purity_circs[0][6][0])

         ┌─────┐ ┌───┐      ┌───┐   ░   ┌───┐     ┌───┐    ┌───┐     ┌─┐   »
qr_0: |0>┤ Sdg ├─┤ H ├───■──┤ Z ├───░───┤ Z ├──■──┤ H ├────┤ S ├─────┤M├───»
         └┬───┬┘ └───┘ ┌─┴─┐└───┘   ░   └───┘┌─┴─┐├───┤┌───┴───┴────┐└╥┘┌─┐»
qr_1: |0>─┤ H ├────────┤ X ├────────░────────┤ X ├┤ H ├┤ Ry(1.5708) ├─╫─┤M├»
          ├───┤ ┌─────┐├───┤     ┌─────┐┌───┐├───┤└─░─┘└───┬───┬────┘ ║ └╥┘»
qr_2: |0>─┤ H ├─┤ Sdg ├┤ H ├──■──┤ Sdg ├┤ H ├┤ Z ├──░──────┤ Z ├──────╫──╫─»
         ┌┴───┴┐└┬───┬┘└───┘┌─┴─┐└┬───┬┘├───┤├───┤  ░      ├───┤      ║  ║ »
qr_3: |0>┤ Sdg ├─┤ H ├──────┤ X ├─┤ H ├─┤ S ├┤ Z ├──░──────┤ Z ├──────╫──╫─»
         └─────┘ └───┘      └───┘ └───┘ └───┘└───┘  ░      └───┘      ║  ║ »
 cr_0: 0 ═════════════════════════════════════════════════════════════╩══╬═»
                                                                         ║ »
 cr_1: 0 ════════════════════════════════════════════════════════════════╩═»
                                                                           »

In [18]:
print (rb_purity_circs[0][7][0])

         ┌─────┐ ┌───┐      ┌───┐   ░   ┌───┐     ┌───┐    ┌───┐        »
qr_0: |0>┤ Sdg ├─┤ H ├───■──┤ Z ├───░───┤ Z ├──■──┤ H ├────┤ S ├────────»
         └┬───┬┘ └───┘ ┌─┴─┐└───┘   ░   └───┘┌─┴─┐├───┤┌───┴───┴────┐┌─┐»
qr_1: |0>─┤ H ├────────┤ X ├────────░────────┤ X ├┤ H ├┤ Ry(1.5708) ├┤M├»
          ├───┤ ┌─────┐├───┤     ┌─────┐┌───┐├───┤└─░─┘└───┬───┬────┘└╥┘»
qr_2: |0>─┤ H ├─┤ Sdg ├┤ H ├──■──┤ Sdg ├┤ H ├┤ Z ├──░──────┤ Z ├──────╫─»
         ┌┴───┴┐└┬───┬┘└───┘┌─┴─┐└┬───┬┘├───┤├───┤  ░      ├───┤      ║ »
qr_3: |0>┤ Sdg ├─┤ H ├──────┤ X ├─┤ H ├─┤ S ├┤ Z ├──░──────┤ Z ├──────╫─»
         └─────┘ └───┘      └───┘ └───┘ └───┘└───┘  ░      └───┘      ║ »
 cr_0: 0 ═════════════════════════════════════════════════════════════╬═»
                                                                      ║ »
 cr_1: 0 ═════════════════════════════════════════════════════════════╩═»
                                                                        »
 cr_2: 0 ═════════════════════════════

In [19]:
print (rb_purity_circs[0][8][0])

         ┌─────┐ ┌───┐      ┌───┐   ░   ┌───┐     ┌───┐    ┌───┐        »
qr_0: |0>┤ Sdg ├─┤ H ├───■──┤ Z ├───░───┤ Z ├──■──┤ H ├────┤ S ├────────»
         └┬───┬┘ └───┘ ┌─┴─┐└───┘   ░   └───┘┌─┴─┐├───┤┌───┴───┴────┐┌─┐»
qr_1: |0>─┤ H ├────────┤ X ├────────░────────┤ X ├┤ H ├┤ Ry(1.5708) ├┤M├»
          ├───┤ ┌─────┐├───┤     ┌─────┐┌───┐├───┤└─░─┘└───┬───┬────┘└╥┘»
qr_2: |0>─┤ H ├─┤ Sdg ├┤ H ├──■──┤ Sdg ├┤ H ├┤ Z ├──░──────┤ Z ├──────╫─»
         ┌┴───┴┐└┬───┬┘└───┘┌─┴─┐└┬───┬┘├───┤├───┤  ░      ├───┤      ║ »
qr_3: |0>┤ Sdg ├─┤ H ├──────┤ X ├─┤ H ├─┤ S ├┤ Z ├──░──────┤ Z ├──────╫─»
         └─────┘ └───┘      └───┘ └───┘ └───┘└───┘  ░      └───┘      ║ »
 cr_0: 0 ═════════════════════════════════════════════════════════════╬═»
                                                                      ║ »
 cr_1: 0 ═════════════════════════════════════════════════════════════╩═»
                                                                        »
 cr_2: 0 ═════════════════════════════

In [20]:
noise_model = NoiseModel()
p1Q = 0.002
p2Q = 0.01
noise_model.add_all_qubit_quantum_error(depolarizing_error(p1Q, 1), 'u2')
noise_model.add_all_qubit_quantum_error(depolarizing_error(2*p1Q, 1), 'u3')
noise_model.add_all_qubit_quantum_error(depolarizing_error(p2Q, 2), 'cx')

In [21]:
#Purity RB circuits - generation using execute - with noise
backend = qiskit.Aer.get_backend('qasm_simulator')
basis_gates = ['u1','u2','u3','cx'] # use U,CX for now
shots = 200
purity_result_list = [[] for d in range(npurity)]
import time
for rb_seed in range(len(rb_purity_circs)):
    for d in range(npurity):
        print('Executing seed %d purity %d length %d'%(rb_seed, d, len(nCliffs)))
        new_circ = rb_purity_circs[rb_seed][d]
        job = qiskit.execute(new_circ, backend=backend, noise_model=noise_model, basis_gates=['u1','u2','u3','cx'])
        purity_result_list[d].append(job.result())
        #for l in range(len(nCliffs)):
        #    print (new_circ[l].name, job.result().get_counts(rb_purity_circs[rb_seed][d][l]))
print("Finished Simulating Purity RB Circuits")

Executing seed 0 purity 0 length 5
Executing seed 0 purity 1 length 5
Executing seed 0 purity 2 length 5
Executing seed 0 purity 3 length 5
Executing seed 0 purity 4 length 5
Executing seed 0 purity 5 length 5
Executing seed 0 purity 6 length 5
Executing seed 0 purity 7 length 5
Executing seed 0 purity 8 length 5
Executing seed 1 purity 0 length 5
Executing seed 1 purity 1 length 5
Executing seed 1 purity 2 length 5
Executing seed 1 purity 3 length 5
Executing seed 1 purity 4 length 5
Executing seed 1 purity 5 length 5
Executing seed 1 purity 6 length 5
Executing seed 1 purity 7 length 5
Executing seed 1 purity 8 length 5
Executing seed 2 purity 0 length 5
Executing seed 2 purity 1 length 5
Executing seed 2 purity 2 length 5
Executing seed 2 purity 3 length 5
Executing seed 2 purity 4 length 5
Executing seed 2 purity 5 length 5
Executing seed 2 purity 6 length 5
Executing seed 2 purity 7 length 5
Executing seed 2 purity 8 length 5
Finished Simulating Purity RB Circuits


In [22]:
# List of results
print (len(purity_result_list))
for i in range(len(purity_result_list)):
    for j in range(len(rb_purity_circs)):
        for l in range(len(nCliffs)):
            print (i, j, l, purity_result_list[i][j].results[l].header.name)

9
0 0 0 rb_purity_ZZ_length_0_seed_0
0 0 1 rb_purity_ZZ_length_1_seed_0
0 0 2 rb_purity_ZZ_length_2_seed_0
0 0 3 rb_purity_ZZ_length_3_seed_0
0 0 4 rb_purity_ZZ_length_4_seed_0
0 1 0 rb_purity_ZZ_length_0_seed_1
0 1 1 rb_purity_ZZ_length_1_seed_1
0 1 2 rb_purity_ZZ_length_2_seed_1
0 1 3 rb_purity_ZZ_length_3_seed_1
0 1 4 rb_purity_ZZ_length_4_seed_1
0 2 0 rb_purity_ZZ_length_0_seed_2
0 2 1 rb_purity_ZZ_length_1_seed_2
0 2 2 rb_purity_ZZ_length_2_seed_2
0 2 3 rb_purity_ZZ_length_3_seed_2
0 2 4 rb_purity_ZZ_length_4_seed_2
1 0 0 rb_purity_XZ_length_0_seed_0
1 0 1 rb_purity_XZ_length_1_seed_0
1 0 2 rb_purity_XZ_length_2_seed_0
1 0 3 rb_purity_XZ_length_3_seed_0
1 0 4 rb_purity_XZ_length_4_seed_0
1 1 0 rb_purity_XZ_length_0_seed_1
1 1 1 rb_purity_XZ_length_1_seed_1
1 1 2 rb_purity_XZ_length_2_seed_1
1 1 3 rb_purity_XZ_length_3_seed_1
1 1 4 rb_purity_XZ_length_4_seed_1
1 2 0 rb_purity_XZ_length_0_seed_2
1 2 1 rb_purity_XZ_length_1_seed_2
1 2 2 rb_purity_XZ_length_2_seed_2
1 2 3 rb_purity_XZ

In [23]:
#Purity RB circuits - generation using compile and run - with noise
backend = qiskit.Aer.get_backend('qasm_simulator')
basis_gates = ['u1','u2','u3','cx'] # use U,CX for now
shots = 200
purity_result_list = [[] for d in range(npurity)]
purity_qobj_list = [[] for d in range(npurity)]
import time
for rb_seed in range(len(rb_purity_circs)):
    for d in range(npurity):
        print('Compiling seed %d purity %d'%(rb_seed, d))
        new_circ = rb_purity_circs[rb_seed][d]
        new_rb_circ_seed = qiskit.compiler.transpile(new_circ, basis_gates=basis_gates)
        qobj = qiskit.compiler.assemble(new_rb_circ_seed, shots=shots)
        print('Simulating seed %d purity %d'%(rb_seed, d))
        job = backend.run(qobj, noise_model=noise_model)
        purity_result_list[d].append(job.result())
        purity_qobj_list[d].append(qobj)    
print("Finished Simulating Purity RB Circuits")

Compiling seed 0 purity 0
Simulating seed 0 purity 0
Compiling seed 0 purity 1
Simulating seed 0 purity 1
Compiling seed 0 purity 2
Simulating seed 0 purity 2
Compiling seed 0 purity 3
Simulating seed 0 purity 3
Compiling seed 0 purity 4
Simulating seed 0 purity 4
Compiling seed 0 purity 5
Simulating seed 0 purity 5
Compiling seed 0 purity 6
Simulating seed 0 purity 6
Compiling seed 0 purity 7
Simulating seed 0 purity 7
Compiling seed 0 purity 8
Simulating seed 0 purity 8
Compiling seed 1 purity 0
Simulating seed 1 purity 0
Compiling seed 1 purity 1
Simulating seed 1 purity 1
Compiling seed 1 purity 2
Simulating seed 1 purity 2
Compiling seed 1 purity 3
Simulating seed 1 purity 3
Compiling seed 1 purity 4
Simulating seed 1 purity 4
Compiling seed 1 purity 5
Simulating seed 1 purity 5
Compiling seed 1 purity 6
Simulating seed 1 purity 6
Compiling seed 1 purity 7
Simulating seed 1 purity 7
Compiling seed 1 purity 8
Simulating seed 1 purity 8
Compiling seed 2 purity 0
Simulating seed 2 pu

In [24]:
# List of results
print (len(purity_result_list))
for i in range(len(purity_result_list)):
    for j in range(len(rb_purity_circs)):
        for l in range(len(nCliffs)):
            print (i, j, l, purity_result_list[i][j].results[l].header.name)

9
0 0 0 rb_purity_ZZ_length_0_seed_0
0 0 1 rb_purity_ZZ_length_1_seed_0
0 0 2 rb_purity_ZZ_length_2_seed_0
0 0 3 rb_purity_ZZ_length_3_seed_0
0 0 4 rb_purity_ZZ_length_4_seed_0
0 1 0 rb_purity_ZZ_length_0_seed_1
0 1 1 rb_purity_ZZ_length_1_seed_1
0 1 2 rb_purity_ZZ_length_2_seed_1
0 1 3 rb_purity_ZZ_length_3_seed_1
0 1 4 rb_purity_ZZ_length_4_seed_1
0 2 0 rb_purity_ZZ_length_0_seed_2
0 2 1 rb_purity_ZZ_length_1_seed_2
0 2 2 rb_purity_ZZ_length_2_seed_2
0 2 3 rb_purity_ZZ_length_3_seed_2
0 2 4 rb_purity_ZZ_length_4_seed_2
1 0 0 rb_purity_XZ_length_0_seed_0
1 0 1 rb_purity_XZ_length_1_seed_0
1 0 2 rb_purity_XZ_length_2_seed_0
1 0 3 rb_purity_XZ_length_3_seed_0
1 0 4 rb_purity_XZ_length_4_seed_0
1 1 0 rb_purity_XZ_length_0_seed_1
1 1 1 rb_purity_XZ_length_1_seed_1
1 1 2 rb_purity_XZ_length_2_seed_1
1 1 3 rb_purity_XZ_length_3_seed_1
1 1 4 rb_purity_XZ_length_4_seed_1
1 2 0 rb_purity_XZ_length_0_seed_2
1 2 1 rb_purity_XZ_length_1_seed_2
1 2 2 rb_purity_XZ_length_2_seed_2
1 2 3 rb_purity_XZ

In [25]:
for i in range(len(purity_result_list)):
    print (i)
    print (rb.RBFitter(purity_result_list[i], xdata, rb_opts['rb_pattern']).raw_data)
    print (rb.RBFitter(purity_result_list[i], xdata, rb_opts['rb_pattern']).ydata)

0
[[[0.975, 0.935, 0.96, 0.935, 0.845], [0.975, 0.89, 0.92, 0.91, 0.84], [0.95, 0.955, 0.895, 0.82, 0.85]], [[0.98, 0.945, 0.895, 0.895, 0.875], [0.985, 0.91, 0.835, 0.865, 0.855], [0.975, 0.95, 0.865, 0.895, 0.835]]]
[{'mean': array([0.96666667, 0.92666667, 0.925     , 0.88833333, 0.845     ]), 'std': array([0.01178511, 0.02718251, 0.02677063, 0.04938511, 0.00408248])}, {'mean': array([0.98 , 0.935, 0.865, 0.885, 0.855]), 'std': array([0.00408248, 0.01779513, 0.0244949 , 0.01414214, 0.01632993])}]
1
[[[0.47, 0.505, 0.435, 0.45, 0.48], [0.515, 0.455, 0.51, 0.505, 0.415], [0.52, 0.54, 0.51, 0.405, 0.485]], [[0.52, 0.49, 0.47, 0.455, 0.465], [0.475, 0.485, 0.44, 0.465, 0.455], [0.555, 0.46, 0.44, 0.48, 0.435]]]
[{'mean': array([0.50166667, 0.5       , 0.485     , 0.45333333, 0.46      ]), 'std': array([0.02248456, 0.03488075, 0.03535534, 0.04089281, 0.03188521])}, {'mean': array([0.51666667, 0.47833333, 0.45      , 0.46666667, 0.45166667]), 'std': array([0.0327448 , 0.01312335, 0.0141421

In [26]:
rbfit_purity = [[] for d in range(npurity)]
for d in range(npurity):
    rbfit_purity[d] = rb.RBFitter(purity_result_list[d], xdata, rb_opts['rb_pattern'])
    print (d, rbfit_purity[d].raw_data)

0 [[[0.975, 0.935, 0.96, 0.935, 0.845], [0.975, 0.89, 0.92, 0.91, 0.84], [0.95, 0.955, 0.895, 0.82, 0.85]], [[0.98, 0.945, 0.895, 0.895, 0.875], [0.985, 0.91, 0.835, 0.865, 0.855], [0.975, 0.95, 0.865, 0.895, 0.835]]]
1 [[[0.47, 0.505, 0.435, 0.45, 0.48], [0.515, 0.455, 0.51, 0.505, 0.415], [0.52, 0.54, 0.51, 0.405, 0.485]], [[0.52, 0.49, 0.47, 0.455, 0.465], [0.475, 0.485, 0.44, 0.465, 0.455], [0.555, 0.46, 0.44, 0.48, 0.435]]]
2 [[[0.53, 0.455, 0.515, 0.475, 0.51], [0.46, 0.5, 0.44, 0.45, 0.43], [0.525, 0.455, 0.435, 0.465, 0.485]], [[0.51, 0.505, 0.46, 0.44, 0.435], [0.49, 0.505, 0.47, 0.46, 0.44], [0.5, 0.505, 0.46, 0.445, 0.46]]]
3 [[[0.505, 0.51, 0.485, 0.505, 0.45], [0.53, 0.48, 0.41, 0.47, 0.48], [0.48, 0.455, 0.54, 0.445, 0.365]], [[0.505, 0.485, 0.44, 0.45, 0.47], [0.435, 0.515, 0.435, 0.425, 0.505], [0.52, 0.495, 0.45, 0.415, 0.51]]]
4 [[[0.265, 0.23, 0.215, 0.22, 0.285], [0.235, 0.245, 0.22, 0.21, 0.24], [0.215, 0.26, 0.26, 0.27, 0.19]], [[0.275, 0.25, 0.24, 0.23, 0.265], [

In [27]:
rbfit_purity1 = rb.PurityRBFitter(purity_result_list, npurity, xdata, rb_opts['rb_pattern'])

In [29]:
print (rbfit_purity1)